In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-prompt-recovery/sample_submission.csv
/kaggle/input/llm-prompt-recovery/train.csv
/kaggle/input/llm-prompt-recovery/test.csv
/kaggle/input/gemma/keras/gemma_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_2b_en/2/assets/tokenizer/vocabulary.spm


In [2]:
!git clone https://github.com/keshavshrikant/MachineLearning.git

Cloning into 'MachineLearning'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 59 (delta 20), reused 43 (delta 12), pack-reused 0
Unpacking objects: 100% (59/59), 5.07 MiB | 9.31 MiB/s, done.


In [3]:
!cd MachineLearning && git checkout kaggle-gemma

Branch 'kaggle-gemma' set up to track remote branch 'kaggle-gemma' from 'origin'.
Switched to a new branch 'kaggle-gemma'


In [4]:
import json
transformations = []
with open("MachineLearning/gpt_generated_tranformations.jsonl", "r") as fin:
    for line in fin:
        instruction_info = json.loads(line)
#         print(instruction_info)
        transformations.append(instruction_info) 

trans_df = pd.DataFrame.from_records(transformations)
trans_df.head()

,instruction,actual_text,rewritten_text
0,"Start with a hypothetical outcome, exploring v...","Hi Delta Airline,\n\nI am Ao Ni, I send this e...",In a parallel universe where my flight from La...
1,Highlight a background detail that was previou...,Swimming in the ocean can be an exhilarating e...,The sound of crashing waves against the shore ...
2,Develop a list of potential spin-off stories f...,Below is a recursive Python function fib(n) th...,"Within the world of the Fibonacci sequence, th..."
3,"Write a letter from one character to another, ...",* Planned fundraising efforts for the next wee...,"Dear Sarah, I wanted to update you on the prog..."
4,Develop a step-by-step guide on how to cultiva...,"Hi Landlord, I wanted to send this message to ...",To cultivate a thriving indoor garden in a sma...


In [5]:
from kaggle_secrets import UserSecretsClient

os.environ["KAGGLE_USERNAME"] = UserSecretsClient().get_secret('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = UserSecretsClient().get_secret('KAGGLE_KEY')

In [6]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.


In [7]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [8]:
import keras
import keras_nlp

2024-03-14 09:39:16.685004: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 09:39:16.685110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 09:39:16.822001: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [47]:
# template = """Provided below is a creative writing paragraph and an instruction for rewriting it in a certain manner. Please follow the instruction to the tee and reqrite the paragraph as specified. 

# Paragraph:
# {para}

# Instruction:
# {instruction}

# Response: {response}
# """

# prompt

"Provided below is a creative writing paragraph and an insturction for rewriting it in a certain manner. Please follow the insturction to the tee and reqrite the paragraph as specified. \n\nParagraph:\nHi Delta Airline,\n\nI am Ao Ni, I send this email regarding a cancelled flight of mine and to request documents to claim my travel insurance. \n\nI just received an email from you that my flight from Las Vegas to New York today at 10 PM was canceled due to mechanical issues. Even though Delta Airline will refund me the cost of flight ticket, the last minute ticket for tomorrow's flight is extremely expensive, almost double the normal price. I have to get back to New York by tomorrow to attend a very important meeting so I have to pay for the ticket, plus one night stay at a hotel for tonight. \n\nWill you reimburse my hotel and ticket? If not, I will have to claim it through my travel insurance and I need two documents to support my case. First, the confirmation of my order. Second, I n

In [11]:
prompt_recovery_template = """Provided below is a creative writing paragraph and an interesting manner in which it has been rewritten. Your job is to figure out the insturction which would have been followed to rewrite the original paragraph in the interesting manner.  

Paragraph:
{para}

Rewritten Paragraph:
{rewritten}

Instruction:
{instruction}
"""


In [12]:
train_df = []
    
for index, row in trans_df.iterrows():

    prompt = prompt_recovery_template.format(
        para=row['actual_text'],
        rewritten=row['rewritten_text'],
        instruction=row['instruction'],
    )

    if len(prompt) < 850:
        train_df.append(prompt)

In [13]:
train_df[0]

'Provided below is a creative writing paragraph and an interesting manner in which it has been rewritten. Your job is to figure out the insturction which would have been followed to rewrite the original paragraph in the interesting manner.  \n\nParagraph:\nAn unicorn, a white horse like animal with a horn in the forehead\n\nRewritten Paragraph:\nMajestic horned steed, Pure white with magic allure, Unicorn in sight\n\nInstruction:\nCraft a haiku summarizing the essence of the narrative in three lines.\n'

In [14]:
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.preprocessor.sequence_length = 850

optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
    beta_1=0.9,
    beta_2=0.999
    )

optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [15]:
gemma_lm.fit(train_df, epochs=20, batch_size=1)

Epoch 1/20


2024-03-14 09:36:50.405199: E external/xla/xla/pjrt/pjrt_stream_executor_client.cc:2732] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 5013541528 bytes.
BufferAssignment OOM Debugging.
BufferAssignment stats:
             parameter allocation:    9.35GiB
              constant allocation:       532B
        maybe_live_out allocation:    9.35GiB
     preallocated temp allocation:    4.67GiB
  preallocated temp fragmentation:     4.6KiB (0.00%)
                 total allocation:   14.02GiB
              total fragmentation:    76.9KiB (0.00%)
Peak buffers:
	Buffer 1:
		Size: 1.95GiB
		Operator: op_name="jit(compiled_train_step)/jit(main)/transpose[permutation=(1, 0)]" source_file="/tmp/ipykernel_34/1668031489.py" source_line=1
		Entry Parameter Subshape: f32[256000,2048]

	Buffer 2:
		Size: 830.08MiB
		XLA Label: fusion
		Shape: f32[1,850,256000]

	Buffer 3:
		Size: 830.08MiB
		Operator: op_name="jit(compiled_train_step)/jit(main)/dot_general[d

XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 5013541528 bytes.
BufferAssignment OOM Debugging.
BufferAssignment stats:
             parameter allocation:    9.35GiB
              constant allocation:       532B
        maybe_live_out allocation:    9.35GiB
     preallocated temp allocation:    4.67GiB
  preallocated temp fragmentation:     4.6KiB (0.00%)
                 total allocation:   14.02GiB
              total fragmentation:    76.9KiB (0.00%)
Peak buffers:
	Buffer 1:
		Size: 1.95GiB
		Operator: op_name="jit(compiled_train_step)/jit(main)/transpose[permutation=(1, 0)]" source_file="/tmp/ipykernel_34/1668031489.py" source_line=1
		Entry Parameter Subshape: f32[256000,2048]
		==========================

	Buffer 2:
		Size: 830.08MiB
		XLA Label: fusion
		Shape: f32[1,850,256000]
		==========================

	Buffer 3:
		Size: 830.08MiB
		Operator: op_name="jit(compiled_train_step)/jit(main)/dot_general[dimension_numbers=(((2,), (0,)), ((), ())) pre

In [ ]:
test_df = pd.read_csv('/kaggle/input/llm-prompt-recovery/test.csv')

prompt = prompt_recovery_template.format(
    para=test['original_text'].iloc[0],
    rewritten=row['rewritten_text'].iloc[0],
    instruction=""
)

response = gemma_lm.generate(prompt,max_length=850)

In [ ]:
response = response.replace(prompt,'')
response

In [49]:
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)